In [1]:
model_name = "dfNGP"
from gpytorch_models import dfNGP

# import configs to we can access the hypers with getattr
import configs
from configs import N_SIDE_INFERENCE
from configs import PATIENCE, MAX_NUM_EPOCHS, WEIGHT_DECAY
from configs import TRACK_EMISSIONS_BOOL
from configs import SCALE_INPUT_region_lower_byrd, SCALE_INPUT_region_mid_byrd, SCALE_INPUT_region_upper_byrd
from configs import REAL_L_RANGE, REAL_NOISE_VAR_RANGE, REAL_OUTPUTSCALE_VAR_RANGE

SCALE_INPUT = {
    "region_lower_byrd": SCALE_INPUT_region_lower_byrd,
    "region_mid_byrd": SCALE_INPUT_region_mid_byrd,
    "region_upper_byrd": SCALE_INPUT_region_upper_byrd,
}

# Reiterating import for visibility
MAX_NUM_EPOCHS = MAX_NUM_EPOCHS
WEIGHT_DECAY = WEIGHT_DECAY
PATIENCE = PATIENCE

# assign model-specific variable
MODEL_LEARNING_RATE = getattr(configs, f"{model_name}_REAL_LEARNING_RATE")
# define results directory
RESULTS_DIR = "results_real/dfNGP_grid_inference"
import os

# basics
import pandas as pd
import torch
import gpytorch

# universals 
from utils import set_seed, make_grid
import gc
import warnings
set_seed(42)

# setting device to GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# overwrite if needed: # device = 'cpu'
print('Using device:', device)
print()

### START TIMING ###
import time
start_time = time.time()  # Start timing after imports

### START TRACKING EXPERIMENT EMISSIONS ###
if TRACK_EMISSIONS_BOOL:
    from codecarbon import EmissionsTracker
    tracker = EmissionsTracker(project_name = "dfNGP_real_grid_inference", output_dir = RESULTS_DIR)
    tracker.start()

#############################
### LOOP 1 - over REGIONS ###
#############################

for region_name in ["region_lower_byrd", "region_mid_byrd", "region_upper_byrd"]:
    SCALE_DOMAIN = SCALE_INPUT[region_name]

    print(f"\nTraining for {region_name.upper()}...")

    # Store metrics for the current region (used for *metrics_summary* report and *metrics_per_run*)
    region_results = []

    ##########################################
    ### x_train & y_train, x_test & x_test ###
    ##########################################

    # define paths based on region_name
    path_to_training_tensor = "data/real_data/" + region_name + "_train_tensor.pt"
    path_to_test_tensor = "data/real_data/" + region_name + "_test_tensor.pt"

    # load and tranpose to have rows as points
    train = torch.load(path_to_training_tensor, weights_only = False).T 
    test = torch.load(path_to_test_tensor, weights_only = False).T

    # The train and test tensors have the following columns:
    # [:, 0] = x
    # [:, 1] = y
    # [:, 2] = surface elevation (s)
    # [:, 3] = ice flux in x direction (u)
    # [:, 4] = ice flux in y direction (v)
    # [:, 5] = ice flux error in x direction (u_err)
    # [:, 6] = ice flux error in y direction (v_err)
    # [:, 7] = source age

    # train
    x_train = train[:, [0, 1]].to(device)
    y_train = train[:, [3, 4]].to(device)

    # test
    x_test = test[:, [0, 1]].to(device)
    y_test = test[:, [3, 4]].to(device)

    # HACK: Scaling helps with numerical stability
    # Units are not in km 
    x_test = x_test * SCALE_DOMAIN
    x_train = x_train * SCALE_DOMAIN

    # Now we combine the training inputs and observations into a single tensor to train our best predictive model
    x_train_grid = torch.concat((x_train, x_test), dim = 0).to(device)
    y_train_grid = torch.concat((y_test, y_train), dim = 0).to(device)

    # Make test aka inference grid
    _, x_test_grid = make_grid(n_side = N_SIDE_INFERENCE)
    x_test_grid = x_test_grid * SCALE_DOMAIN # scale grid to match training data
    x_test_grid.requires_grad_(True) # need gradients for divergence field

    # NOTE: Here we estimate the noise variance 

    # Print train details
    print(f"=== {region_name.upper()} ===")
    print(f"Training inputs shape: {x_train_grid.shape}")
    print(f"Training observations shape: {y_train_grid.shape}")
    print(f"Training inputs dtype: {x_train_grid.dtype}")
    print()

    # Print test details
    print(f"=== {region_name.upper()} ===")
    print(f"Test inputs shape: {x_test_grid.shape}")
    # NOTE: No test observations, only inputs
    print(f"Test inputs dtype: {x_test_grid.dtype}")
    print()

    #####################
    ### SPECFIY MODEL ###
    #####################

    print(f"\n--- Training starts ---")

    # Initialise the likelihood for the GP model (estimates noise)
    # NOTE: we use a multitask likelihood for the dfNGP model but with a global noise term
    likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(
        num_tasks = 2,
        has_global_noise = True, 
        has_task_noise = False, # HACK: This still needs to be manually turned off
        ).to(device)

    # NOTE: This was needed
    x_train_grid = x_train_grid.clone().detach().requires_grad_(True)

    # Use all data to train the grid model
    model = dfNGP(
        x_train_grid,
        y_train_grid, 
        likelihood
        ).to(device)
    
    # Initialise hypers
    # NOTE: Alternative is to start with best hypers from previously trained model
    # Overwrite default lengthscale hyperparameter initialisation because we have a different input scale.
    model.base_kernel.lengthscale = torch.empty([1, 2], device = device).uniform_( * REAL_L_RANGE)
    # Overwrite default outputscale variance initialisation.
    model.covar_module.outputscale = torch.empty(1, device = device).uniform_( * REAL_OUTPUTSCALE_VAR_RANGE)
    # Overwrite default noise variance initialisation because this is real noisy data.
    model.likelihood.noise = torch.empty(1, device = device).uniform_( * REAL_NOISE_VAR_RANGE)

    # NOTE: This part is different from dfGP
    optimizer = torch.optim.AdamW([
        {"params": model.mean_module.parameters(), 
            "weight_decay": WEIGHT_DECAY, "lr": MODEL_LEARNING_RATE * 0.2},
        {"params": list(model.covar_module.parameters()) + list(model.likelihood.parameters()), 
            "weight_decay":  WEIGHT_DECAY, "lr": MODEL_LEARNING_RATE},
        ])
    
    # Use ExactMarginalLogLikelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    # Early stopping variables
    best_loss = float('inf')
    # counter starts at 0
    epochs_no_improve = 0

    ############################
    ### LOOP 2 - over EPOCHS ###
    ############################
    print("\nStart Training")

    for epoch in range(MAX_NUM_EPOCHS):

        # Set to train
        model.train()
        likelihood.train()

        # Do a step
        optimizer.zero_grad()

        x_train_grid = x_train_grid.clone().detach().requires_grad_(True)
        train_pred_dist = model(x_train_grid.to(device))
        loss = - mll(train_pred_dist, y_train_grid.to(device))
        loss.backward()
        optimizer.step()

        if epoch % 50 == 0:
            print(f"{region_name} {model_name} Epoch {epoch + 1}/{MAX_NUM_EPOCHS}, Training Loss (NLML): {loss:.4f}")
            print(f"Lengthscale x1: {model.base_kernel.lengthscale[:,0].cpu().detach().numpy().item():.3f}")
            print(f"Lengthscale x2: {model.base_kernel.lengthscale[:,1].cpu().detach().numpy().item():.3f}")
            print(f"Outputscale: {model.covar_module.outputscale.cpu().detach().numpy().item():.3f}")
            print(f"Noise: {model.likelihood.noise.cpu().detach().numpy().item():3f}")

        if loss < best_loss:
            best_loss = loss
            # reset counter if loss improves
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= PATIENCE:
            print(f"Early stopping triggered after {epoch + 1} epochs.")
            # exit epoch loop
            break
    
    #################
    ### INFERENCE ###
    #################

    model.eval()
    likelihood.eval()

    x_test_grid.requires_grad_(True)

    dist_grid = model(x_test_grid.to(device))
    pred_dist_grid = likelihood(dist_grid)

    torch.save(pred_dist_grid.mean, f"{RESULTS_DIR}/{region_name}_{model_name}_grid_mean_predictions.pt")
    torch.save(pred_dist_grid.covariance_matrix, f"{RESULTS_DIR}/{region_name}_{model_name}_grid_covar_predictions.pt")
    torch.save(dist_grid.covariance_matrix, f"{RESULTS_DIR}/{region_name}_{model_name}_grid_latent_covar_predictions.pt")

#################################
### END LOOP 1 - over REGIONS ###
#################################

# also end emission tracking. Will be saved as emissions.csv
if TRACK_EMISSIONS_BOOL:
    tracker.stop()

[codecarbon WARNING @ 12:19:43] Multiple instances of codecarbon are allowed to run at the same time.


Using device: cuda



[codecarbon INFO @ 12:19:43] [setup] RAM Tracking...
[codecarbon INFO @ 12:19:43] [setup] CPU Tracking...
[codecarbon WARNING @ 12:19:44] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 12:19:44] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU E5-2630 v4 @ 2.20GHz
[codecarbon WARNING @ 12:19:44] No CPU tracking mode found. Falling back on CPU load mode.
[codecarbon INFO @ 12:19:44] [setup] GPU Tracking...
[codecarbon INFO @ 12:19:44] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 12:19:44] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: cpu_load
                GPU Tracking Method: pynvml
            
[codecarbon INFO @ 12:19:44] >>> Tracker's metadata:
[codecarbon INFO @ 12:19:44]   Platform system: Linux-5.15.0-139


Training for REGION_LOWER_BYRD...
=== REGION_LOWER_BYRD ===
Training inputs shape: torch.Size([829, 2])
Training observations shape: torch.Size([829, 2])
Training inputs dtype: torch.float32

=== REGION_LOWER_BYRD ===
Test inputs shape: torch.Size([900, 2])
Test inputs dtype: torch.float32


--- Training starts ---

Start Training
region_lower_byrd dfNGP Epoch 1/2000, Training Loss (NLML): 20.0303
Lengthscale x1: 7.184
Lengthscale x2: 5.353
Outputscale: 1.199
Noise: 0.029591
region_lower_byrd dfNGP Epoch 51/2000, Training Loss (NLML): 14.9404
Lengthscale x1: 6.939
Lengthscale x2: 5.095
Outputscale: 1.361
Noise: 0.037530
region_lower_byrd dfNGP Epoch 101/2000, Training Loss (NLML): 11.3857
Lengthscale x1: 6.714
Lengthscale x2: 4.796
Outputscale: 1.449
Noise: 0.046299
region_lower_byrd dfNGP Epoch 151/2000, Training Loss (NLML): 9.1912
Lengthscale x1: 6.526
Lengthscale x2: 4.546
Outputscale: 1.489
Noise: 0.055629
region_lower_byrd dfNGP Epoch 201/2000, Training Loss (NLML): 7.6001
Lengt

[codecarbon INFO @ 12:20:03] Energy consumed for RAM : 0.000086 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 12:20:04] Delta energy consumed for CPU with cpu_load : 0.000073 kWh, power : 17.0 W
[codecarbon INFO @ 12:20:04] Energy consumed for All CPU : 0.000073 kWh
[codecarbon INFO @ 12:20:04] Energy consumed for all GPUs : 0.000332 kWh. Total GPU Power : 74.56804072164141 W
[codecarbon INFO @ 12:20:04] 0.000491 kWh of electricity used since the beginning.


...

[codecarbon INFO @ 12:23:19] Delta energy consumed for CPU with cpu_load : 0.000068 kWh, power : 17.0 W
[codecarbon INFO @ 12:23:19] Energy consumed for All CPU : 0.000963 kWh
[codecarbon INFO @ 12:23:19] Energy consumed for all GPUs : 0.004547 kWh. Total GPU Power : 77.29906584196952 W
[codecarbon INFO @ 12:23:19] 0.006643 kWh of electricity used since the beginning.


region_upper_byrd dfNGP Epoch 1601/2000, Training Loss (NLML): 1.5556
Lengthscale x1: 2.764
Lengthscale x2: 4.357
Outputscale: 2.237
Noise: 0.375579
region_upper_byrd dfNGP Epoch 1651/2000, Training Loss (NLML): 1.5418
Lengthscale x1: 2.735
Lengthscale x2: 4.339
Outputscale: 2.252
Noise: 0.386112
region_upper_byrd dfNGP Epoch 1701/2000, Training Loss (NLML): 1.5245
Lengthscale x1: 2.708
Lengthscale x2: 4.322
Outputscale: 2.266
Noise: 0.396537
region_upper_byrd dfNGP Epoch 1751/2000, Training Loss (NLML): 1.5112
Lengthscale x1: 2.684
Lengthscale x2: 4.308
Outputscale: 2.280
Noise: 0.406782
region_upper_byrd dfNGP Epoch 1801/2000, Training Loss (NLML): 1.4933
Lengthscale x1: 2.663
Lengthscale x2: 4.296
Outputscale: 2.293
Noise: 0.416875
region_upper_byrd dfNGP Epoch 1851/2000, Training Loss (NLML): 1.4818
Lengthscale x1: 2.644
Lengthscale x2: 4.287
Outputscale: 2.306
Noise: 0.426810
region_upper_byrd dfNGP Epoch 1901/2000, Training Loss (NLML): 1.4756
Lengthscale x1: 2.627
Lengthscale x2

[codecarbon INFO @ 12:23:33] Energy consumed for RAM : 0.001213 kWh. RAM Power : 20.0 W


region_upper_byrd dfNGP Epoch 1951/2000, Training Loss (NLML): 1.4601
Lengthscale x1: 2.613
Lengthscale x2: 4.275
Outputscale: 2.331
Noise: 0.446139


[codecarbon INFO @ 12:23:34] Delta energy consumed for CPU with cpu_load : 0.000069 kWh, power : 17.1275 W
[codecarbon INFO @ 12:23:34] Energy consumed for All CPU : 0.001032 kWh
[codecarbon INFO @ 12:23:34] Energy consumed for all GPUs : 0.004871 kWh. Total GPU Power : 77.64778546564197 W
[codecarbon INFO @ 12:23:34] 0.007116 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:23:35] Energy consumed for RAM : 0.001223 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 12:23:36] Delta energy consumed for CPU with cpu_load : 0.000009 kWh, power : 18.4025 W
[codecarbon INFO @ 12:23:36] Energy consumed for All CPU : 0.001041 kWh
[codecarbon INFO @ 12:23:36] Energy consumed for all GPUs : 0.004919 kWh. Total GPU Power : 78.29739211425156 W
[codecarbon INFO @ 12:23:36] 0.007182 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:23:36] 0.017187 g.CO2eq/s mean an estimation of 542.0119306886696 kg.CO2eq/year
